In [1]:
import pandas as pd
import requests
import re
import time
import random
import datetime
import matplotlib.pyplot as plt
import FinanceDataReader as fdr 
from datetime import timedelta
from datetime import datetime
import requests 
from bs4 import BeautifulSoup 
import urllib.request as r
from pykrx import stock

In [2]:
# 오늘날 코스피에 상장되어 있는 주식의 이름과 티커 수집 
today = datetime.today().strftime("%Y%m%d") 
ticker_list = stock.get_market_ticker_list(date = today, market="KOSPI") 
symbol_list = [] 
for ticker in ticker_list: 
  symbol = stock.get_market_ticker_name(ticker) 
  symbol_list.append(symbol) 

df_우 = [s for s in symbol_list if s.endswith('우') | s.endswith('우B') | s.endswith('우(전환)')]
res = [i for i in symbol_list if i not in df_우]
print(len(res))

820


In [3]:
class Company:
    def __init__(self, Name, Code):
        self.Name = Name
        self.Code = Code
        self.LoadStockSummary(Code)

    def LoadStockSummary(self, ticker):
        url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A" + ticker + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=Y&stkGb=701"
        response = r.urlopen(url)
        soup = BeautifulSoup(response, 'html.parser')
    
        # get PER PBR ROE
        tmp = soup.find('div', {'class': 'corp_group2'})
        tmp2 = tmp.findAll('dd')
        self.PER = tmp2[1].contents
        self.PER12M = tmp2[3].contents
        self.PERIndustry = tmp2[5].contents
        self.PBR = tmp2[7].contents
        self.DividendRatio = tmp2[9].contents

        # get Summary
        for anchor in soup.select(".um_bssummary"):
            pass
        self.Summary = anchor.get_text()

    def print_info(self):
        print("Name: ", self.Name)
        print("Code: ", self.Code)
        print("Summary: ", self.Summary)
        print("PER: ", self.PER)
        print("PER12M: ", self.PER12M)
        print("PERIndustry: ", self.PERIndustry)
        print("PBR: ", self.PBR)
        print("DividendRatio: ", self.DividendRatio)


def run():

    Name = "CJ대한통운"
    Code = "000120"
    company = Company(Name, Code)
    company.print_info()


if __name__ == "__main__":
    run()

Name:  CJ대한통운
Code:  000120
Summary:  
택배가 인상, 비대면 확산 이익 개선

1930년 11월에 설립되어 CL사업부문, 택배사업부문, 글로벌사업부문, 건설사업부문을 주된 사업부문으로 영위함. 최근 급성장하고 있는 이커머스 시장에 대응하기 위해 독자적인 융합형 'CJ대한통운 e-풀필먼트' 서비스를 개시함. 매출구성은 글로벌사업부문 39.41%, 택배사업부문 31.75%, CL사업부문 23.57%, 건설사업부문 5.27%로 이루어져 있음.2021년 12월 전년동기 대비 연결기준 매출액은 5.2% 증가, 영업이익은 5.7% 증가, 당기순이익은 55.5% 감소. 언택트 소비 트렌드 확산에 따른 택배부문 고성장에 따라 매출은 전년 대비 5.2% 성장한 11조 3,436억원, 영업이익은 5.7% 증가한 3,438억원을 달성. 전국적 택배 Network 및 풀필먼트 역량을 활용하고 향후 디지털 물류 역량 확보를 위해 노력.


PER:  ['52.30']
PER12M:  ['14.61']
PERIndustry:  ['7.78']
PBR:  ['0.74']
DividendRatio:  ['-']


In [28]:
url_1 = "http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A000120&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"
aes = requests.get(url_1, headers={'User-agent':'Mozilla/5.0'})
sof = pd.read_html(aes.text)

In [29]:
sof[0]

,IFRS(연결),2018/12,2019/12,2020/12,2021/12,2022/03
0,안정성비율,안정성비율,안정성비율,안정성비율,안정성비율,안정성비율
1,유동비율계산에 참여한 계정 펼치기,83.4,92.6,90.4,95.9,97.9
2,당좌비율계산에 참여한 계정 펼치기,82.6,91.7,89.5,94.9,96.8
3,부채비율계산에 참여한 계정 펼치기,151.3,149.2,138.7,123.9,126.0
4,유보율계산에 참여한 계정 펼치기,2627.4,2923.4,3073.1,3272.4,3282.0
5,순차입금비율계산에 참여한 계정 펼치기,85.4,76.1,67.6,61.1,67.6
6,이자보상배율계산에 참여한 계정 펼치기,2.8,2.4,2.8,3.4,NaN
7,자기자본비율계산에 참여한 계정 펼치기,39.8,40.1,41.9,44.7,44.3
8,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율
9,매출액증가율계산에 참여한 계정 펼치기,29.7,13.0,3.5,5.2,6.1


In [25]:
sof[1]

,IFRS(연결),2021/03,2021/06,2021/09,2021/12,2022/03
0,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율
1,매출액증가율계산에 참여한 계정 펼치기,18.2,20.2,10.5,24.4,19.0
2,영업이익증가율계산에 참여한 계정 펼치기,45.5,54.3,28.0,53.3,50.5
3,EBITDA증가율계산에 참여한 계정 펼치기,24.4,32.1,23.2,38.7,38.8
4,EPS증가율계산에 참여한 계정 펼치기,45.1,72.2,30.1,65.1,56.9
5,수익성비율,수익성비율,수익성비율,수익성비율,수익성비율,수익성비율
6,세전계속사업이익률계산에 참여한 계정 펼치기,14.9,20.2,22.1,18.8,19.4
7,영업이익률계산에 참여한 계정 펼치기,14.4,19.7,21.4,18.1,18.2
8,EBITDA마진율계산에 참여한 계정 펼치기,26.3,32.3,33.3,30.7,30.7


In [5]:
URL = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=Y&stkGb=701"
res = requests.get(URL, headers={'User-agent':'Mozilla/5.0'})
dof = pd.read_html(res.text)


In [6]:
dof[0]

,0,1,2,3
0,종가/ 전일대비,"66,500/ -1,400",거래량,15482576
1,52주.최고가/ 최저가,"82,900/ 64,800",거래대금(억원),10355
2,수익률(1M/ 3M/ 6M/ 1Y),-0.75/ -6.99/ -11.10/ -16.56,외국인 보유비중,50.69
3,"시가총액(상장예정포함,억원)",4457054,베타(1년),0.97393
4,"시가총액(보통주,억원)",3969905,액면가,100
5,NaN,NaN,NaN,NaN
6,발행주식수(보통주/ 우선주),"5,969,782,550/ 822,886,700",유동주식수/비율(보통주),"4,490,908,512 / 75.23"


In [7]:
dof[1]

,"확정실적(영업이익, 억원)","예상실적(영업이익, 억원)",3개월전예상실적대비(%),전년동기대비(%)
0,141214,NaN,5.45,50.5


In [8]:
dof[2]

,운용사명,보유수량,시가평가액,상장주식수내비중,운용사내비중
0,삼성자산운용,52051.87,36228.10,0.87,6.85
1,미래에셋자산운용,32779.98,22814.86,0.55,4.30
2,한국투자신탁운용,10899.85,7586.30,0.18,4.00
3,케이비자산운용,10681.00,7433.97,0.18,3.35
4,엔에이치아문디자산운용,9623.38,6697.87,0.16,4.93
5,신영자산운용,8418.29,5859.13,0.14,14.65
6,교보악사자산운용,7218.02,5023.74,0.12,6.51
7,신한자산운용,5560.79,3870.31,0.09,2.32
8,한화자산운용,5327.80,3708.15,0.09,4.29
9,키움투자자산운용,5220.31,3633.34,0.09,3.31


In [9]:
dof[3]

,항목,보통주,지분율,최종변동일
0,삼성생명보험(외 16인),1241174813,20.79,2022/04/29
1,국민연금공단,518509578,8.69,2021/10/29
2,BlackRock Fund Advisors(외 15인),300391061,5.03,2019/01/28
3,이병철,408650,0.01,2018/05/04
4,서병삼,70300,0.00,2018/05/04
5,윤주화,66000,0.00,2018/05/04


In [10]:
dof[4]

,주주구분,대표주주수,보통주,지분율,최종변동일
0,최대주주등 (본인+특별관계자),1.0,1.241175e+09,20.79,2022/04/29
1,10%이상주주 (본인+특별관계자),NaN,NaN,NaN,NaN
2,5%이상주주 (본인+특별관계자),2.0,8.189006e+08,13.72,2021/10/29
3,"임원 (5%미만 중, 임원인자)",290.0,2.239826e+06,0.04,2022/05/18
4,자기주식 (자사주+자사주신탁),NaN,NaN,NaN,NaN
5,우리사주조합,NaN,NaN,NaN,NaN


In [11]:
dof[5]

,KIS,KR,NICE
0,관련 데이터가 없습니다.,관련 데이터가 없습니다.,관련 데이터가 없습니다.


In [12]:
dof[6]

,KIS,KR,NICE
0,AAA [2004/05/25],AAA [2004/06/17],AAA [2004/06/01]


In [21]:
dof[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   투자의견    1 non-null      float64
 1   목표주가    1 non-null      int64  
 2   EPS     1 non-null      int64  
 3   PER     1 non-null      float64
 4   추정기관수   1 non-null      int64  
dtypes: float64(2), int64(3)
memory usage: 168.0 bytes


In [14]:
dof[8]

,구분,삼성전자,"코스피 전기,전자",KOSPI
0,시가총액,3969905.00,7563434.00,20493970.00
1,매출액,2796048.00,5011923.00,26395032.00
2,영업이익,516339.00,759610.00,2424536.00
3,EPS(원),5777.00,24967.12,8359.66
4,PER,13.55,12.84,11.07
5,EV/EBITDA,6.23,6.19,6.83
6,ROE,13.92,13.54,10.85
7,배당수익률,1.84,1.48,1.77
8,베타(1년),1.06,1.14,1.00


In [15]:
dof[9]

,구분,삼성전자,"코스피 전기,전자",KOSPI
0,시가총액,3969905.00,7563434.00,20493970.00
1,매출액,1997447.00,3583900.00,15249591.00
2,영업이익,319932.00,496883.00,1348053.00
3,EPS(원),4559.00,19739.27,5266.01
4,PER,17.17,16.24,17.58
5,EV/EBITDA,9.64,9.28,10.23
6,ROE,16.45,14.63,8.48
7,배당수익률,1.84,1.48,1.77
8,베타(1년),1.06,1.14,1.00


In [22]:
dof[10]

IFRS(연결)      Annual                                     Net Quarter  \
      IFRS(연결)     2019/12     2020/12     2021/12  2022/12(E)     2021/09   
0          매출액  2304009.00  2368070.00  2796048.00  3267821.00   739792.00   
1         영업이익   277685.00   359939.00   516339.00   635904.00   158175.00   
2   영업이익(발표기준)   277685.00   359939.00   516339.00         NaN   158175.00   
3        당기순이익   217389.00   264078.00   399075.00   489530.00   122933.00   
4      지배주주순이익   215051.00   260908.00   392438.00   481056.00   120572.00   
5     비지배주주순이익     2338.00     3170.00     6637.00         NaN     2361.00   
6         자산총계  3525645.00  3782357.00  4266212.00  4715273.00  4104207.00   
7         부채총계   896841.00  1022877.00  1217212.00  1265283.00  1136546.00   
8         자본총계  2628804.00  2759480.00  3048999.00  3449989.00  2967661.00   
9       지배주주지분  2549155.00  2676703.00  2962377.00  3359898.00  2883312.00   
10     비지배주주지분    79649.00    82777.00    86622.00    90091.00    84349.00   
11         자본금     8975.00     8975.00     8975.00     8979.00     8975.00   
12        부채비율       34.12       37.07       39.92       36.67       38.30   
13         유보율    28302.40    29723.53    32906.47         NaN    32025.54   
14       영업이익률       12.05       15.20       18.47       19.46       21.38   
15    지배주주순이익률        9.33       11.02       14.04       14.72       16.30   
16         ROA        6.28        7.23        9.92       10.90       12.37   
17         ROE        8.69        9.99       13.92       15.22       17.15   
18      EPS(원)     3166.00     3841.00     5777.00     7082.00     1775.00   
19      BPS(원)    37528.00    39406.00    43611.00    49464.00    42447.00   
20      DPS(원)     1416.00     2994.00     1444.00     1613.00      361.00   
21         PER       17.63       21.09       13.55        9.39         NaN   
22         PBR        1.49        2.06        1.80        1.34        1.75   
23       발행주식수  5969783.00  5969783.00  5969783.00         NaN  5969783.00   
24       배당수익률        2.54        3.70        1.84         NaN        0.49   

                                       
       2021/12     2022/03 2022/06(E)  
0    765655.00   777815.00  786748.00  
1    138667.00   141214.00  152932.00  
2    138667.00   141214.00        NaN  
3    108379.00   113246.00  116940.00  
4    106431.00   111291.00  111841.00  
5      1948.00     1955.00        NaN  
6   4266212.00  4393270.00        NaN  
7   1217212.00  1240360.00        NaN  
8   3048999.00  3152909.00        NaN  
9   2962377.00  3063919.00        NaN  
10    86622.00    88990.00        NaN  
11     8975.00     8975.00        NaN  
12       39.92       39.34        NaN  
13    32906.47    34037.84        NaN  
14       18.11       18.16      19.44  
15       13.90       14.31      14.22  
16       10.36       10.46      10.65  
17       14.57       14.77      14.60  
18     1567.00     1638.00    1647.00  
19    43611.00    45106.00        NaN  
20      361.00      361.00        NaN  
21         NaN         NaN        NaN  
22        1.80        1.54        NaN  
23  5969783.00  5969783.00        NaN  
24        0.46        0.52        NaN

In [16]:
dof[10].iloc[0][1]

2304009.0

In [17]:
dof[10].iloc[0][4]

3267821.0

In [18]:
credit_df = pd.read_csv('./기업_신용등급.csv')
credit_df

FileNotFoundError: [Errno 2] No such file or directory: './기업_신용등급.csv'

In [ ]:
ac = credit_df['업종']

In [ ]:
credit_df['업종'].value_counts()

코스피 화학       67
코스피 서비스업     54
코스피 금융업      53
코스피 유통업      48
코스피 운수장비     39
코스피 전기,전자    37
코스피 철강및금속    33
코스피 음식료품     32
코스피 의약품      32
코스피 기계       24
코스피 건설업      24
코스피 비금속광물    20
코스피 운수창고     20
코스피 섬유,의복    19
코스피 증권       19
코스피 종이,목재    13
코스피 보험       10
코스피 전기가스업     9
코스피 제조업       7
코스피 통신업       4
코스피 은행        3
코스피 의료정밀      2
Name: 업종, dtype: int64

In [ ]:
len(credit_df['업종'].unique())

22

In [ ]:
data_df = pd.read_csv('네이버_재무제표_원본.csv', encoding='cp949')
data_df

,Unnamed: 0,종목코드,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,0,95570,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
1,1,6840,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0
2,2,27410,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0
3,3,282330,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0
4,4,138930,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,815,5870,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0
816,816,79980,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0
817,817,5010,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0
818,818,540,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0


In [ ]:
data_df.drop(['Unnamed: 0', '종목코드'], axis=1, inplace=True)

In [ ]:
data_df['동일업종PER'].unique()

array([ 10.83,   8.54,   6.23,   5.29,   5.7 ,   7.92,  46.81,  10.81,
        11.99,   4.14,  31.23,   3.3 ,   6.72,  10.3 ,   8.48,   7.37,
        12.87,   9.58,   4.12,   6.35,    nan, -56.47,  13.02,   6.59,
         6.21,   3.33,  -6.79,  80.19,  81.13,  22.33,  22.32,  18.96,
        10.36,  28.73,  77.14,   7.79,   7.38,  11.93,  16.35,   5.04,
        28.77,  75.58,   5.95,  19.97,  17.94,  31.16,  20.95,  20.57,
         4.63,  10.92,  47.07,  12.77,  18.82,   7.98,  12.62,  -6.8 ,
        28.27,   4.79,  20.84,  19.8 ,  24.8 ,  13.8 ,  23.29,  17.04,
        21.61,  11.27, -26.63,  39.75,  16.3 ,   4.82,  30.19,   9.62,
        13.95, -28.73,  18.76,  21.75,  21.  , -31.67,   6.99,   5.74,
        11.15, -28.75, -43.  ,  16.19,   3.49,  17.55,  10.89,  16.54,
         4.19,  30.53, -56.75,  18.57,  18.51, -53.89,  61.65, 130.92,
        29.89,  56.04,  21.79,  20.38,  -3.72,  22.6 ,  18.64,  16.56])

In [ ]:
data_df

,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
1,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0
2,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0
3,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0
4,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0
816,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0
817,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0
818,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0


In [ ]:
sa = data_df['종목명'].tolist()
sb = credit_df['종목명'].tolist()

In [ ]:
sc = [i for i in sa if i in sb]



In [ ]:
len(sc)

567

In [ ]:
real_df = data_df[data_df['종목명'].isin(sc)]

In [ ]:
real_df.isnull().sum()

종목명                0
PER              118
동일업종PER            5
추정PER            376
PBR                6
ROE              118
배당수익률            167
PER10이하주           0
PBR1이하주            0
ROE12이상주           0
동일업종대비PER하위주       0
PER대비추정PER하위주      0
배당수익률2이상주          0
2346만족             0
12346만족            0
123456만족           0
dtype: int64

In [ ]:
real_df.reset_index(inplace=True)


In [ ]:
real_df.insert(2, '업종', ac)

In [ ]:
aa = real_df[['index', '업종']]

In [ ]:
aa

,index,업종
0,0,코스피 서비스업
1,2,코스피 금융업
2,4,코스피 금융업
3,6,코스피 금융업
4,7,코스피 서비스업
...,...,...
562,814,코스피 화학
563,815,코스피 화학
564,816,코스피 유통업
565,818,"코스피 전기,전자"


In [ ]:
real_df.isnull().sum()

index              0
종목명                0
업종                 0
PER              118
동일업종PER            5
추정PER            376
PBR                6
ROE              118
배당수익률            167
PER10이하주           0
PBR1이하주            0
ROE12이상주           0
동일업종대비PER하위주       0
PER대비추정PER하위주      0
배당수익률2이상주          0
2346만족             0
12346만족            0
123456만족           0
dtype: int64

In [ ]:
credit_df.isnull().sum()

종목명             0
업종              0
기업어음(KIS)     416
기업어음(KR)      405
기업어음(NICE)    376
회사채(KIS)      274
회사채(KR)       270
회사채(NICE)     246
dtype: int64

In [ ]:
ac

0       코스피 서비스업
1        코스피 금융업
2        코스피 금융업
3        코스피 금융업
4       코스피 서비스업
         ...    
564      코스피 유통업
565    코스피 전기,전자
566       코스피 화학
567       코스피 보험
568     코스피 운수창고
Name: 업종, Length: 569, dtype: object

In [ ]:
aa.set_index('index', inplace=True)

In [ ]:
aa

,업종
index,
0,코스피 서비스업
2,코스피 금융업
4,코스피 금융업
6,코스피 금융업
7,코스피 서비스업
...,...
814,코스피 화학
815,코스피 화학
816,코스피 유통업


In [ ]:
import numpy as np

In [ ]:
data_df['업종'] = np.nan

In [ ]:
data_df

,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족,업종
0,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0,NaN
1,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0,NaN
2,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0,NaN
3,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0,NaN
4,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0,NaN
816,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0,NaN
817,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0,NaN
818,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0,NaN


In [ ]:
aa.reset_index(inplace=True)
aa

,index,업종
0,0,코스피 서비스업
1,2,코스피 금융업
2,4,코스피 금융업
3,6,코스피 금융업
4,7,코스피 서비스업
...,...,...
562,814,코스피 화학
563,815,코스피 화학
564,816,코스피 유통업
565,818,"코스피 전기,전자"


In [ ]:
aa.drop('level_0', axis=1, inplace=True)

KeyError: "['level_0'] not found in axis"

In [ ]:
aa

,index,업종
0,0,코스피 서비스업
1,2,코스피 금융업
2,4,코스피 금융업
3,6,코스피 금융업
4,7,코스피 서비스업
...,...,...
562,814,코스피 화학
563,815,코스피 화학
564,816,코스피 유통업
565,818,"코스피 전기,전자"


In [ ]:
data_df['업종'] = aa

In [ ]:
pork = credit_df['업종'] 

In [ ]:
pork

0       코스피 서비스업
1        코스피 금융업
2        코스피 금융업
3        코스피 금융업
4       코스피 서비스업
         ...    
564      코스피 유통업
565    코스피 전기,전자
566       코스피 화학
567       코스피 보험
568     코스피 운수창고
Name: 업종, Length: 569, dtype: object

In [ ]:
aa

,index,업종
0,0,코스피 서비스업
1,2,코스피 금융업
2,4,코스피 금융업
3,6,코스피 금융업
4,7,코스피 서비스업
...,...,...
562,814,코스피 화학
563,815,코스피 화학
564,816,코스피 유통업
565,818,"코스피 전기,전자"


In [ ]:
data_df

,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족,업종
0,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0,코스피 서비스업
1,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0,NaN
2,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0,코스피 금융업
3,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0,NaN
4,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0,코스피 금융업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0,코스피 화학
816,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0,코스피 유통업
817,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0,NaN
818,흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0,"코스피 전기,전자"


In [ ]:
col1=data_df.columns[-1:].to_list()
col2=data_df.columns[:-1].to_list()
new_col=col1+col2
data_df=data_df[new_col]

In [ ]:
data_df

,업종,종목명,PER,동일업종PER,추정PER,PBR,ROE,배당수익률,PER10이하주,PBR1이하주,ROE12이상주,동일업종대비PER하위주,PER대비추정PER하위주,배당수익률2이상주,2346만족,12346만족,123456만족
0,코스피 서비스업,AJ네트웍스,4.16,10.83,NaN,0.84,0.20,3.96,1,1,1,1,0,1,1,1,0
1,NaN,AK홀딩스,NaN,8.54,NaN,0.43,NaN,1.02,0,1,0,0,0,0,0,0,0
2,코스피 금융업,BGF,7.45,6.23,NaN,0.31,0.04,2.16,1,1,0,0,0,1,0,0,0
3,NaN,BGF리테일,21.95,6.23,17.0,4.00,0.18,1.60,0,0,1,0,1,0,0,0,0
4,코스피 금융업,BNK금융지주,3.18,5.29,3.0,0.27,0.08,7.25,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,코스피 화학,휴니드,9.88,31.16,NaN,0.90,0.09,NaN,1,1,0,1,0,0,0,0,0
816,코스피 유통업,휴비스,NaN,4.79,NaN,0.52,NaN,4.20,0,1,0,0,0,1,0,0,0
817,NaN,휴스틸,4.68,4.14,NaN,0.29,0.06,3.46,1,1,0,0,0,1,0,0,0
818,"코스피 전기,전자",흥국화재,3.89,6.72,NaN,0.35,0.09,NaN,1,1,0,1,0,0,0,0,0


In [ ]:
ab = aa['index']

In [ ]:
credit_df.isnull().sum()

종목명             0
업종              0
기업어음(KIS)     416
기업어음(KR)      405
기업어음(NICE)    376
회사채(KIS)      274
회사채(KR)       270
회사채(NICE)     246
dtype: int64

In [ ]:
credit_df.set_index(ab)

ValueError: Length mismatch: Expected 569 rows, received array of length 567